# Process data from PDC assays
## Notes
DO 1-26-2026 I'm re-creating this file with the help of claude code

## Analysis plan
* Load the "Enzyme_assay_metadata" spreadsheet and identify the assays we want to process
* Find all of the .csv files with PDC enzyme assay data
* For each csv file:
  * Add filename information
  * Measure initial pyruvate
    * Determine the expected initial pyruvate concentration (Pyruvate_mM) and Blank_time_s from the Enzyme_assay_metadata dataframe
    * Calculate the pyruvate concentration using the _calculate_blank_pyruvate() function imported from the "Compiling_spectrum_data.ipynb" notebook in the "Spectrum files from Agilent spec" folder
    * If the difference from the expected pyruvate concentration is >50%, throw a warning and use the expected pyruvate concentration instead (note, it might make senese to update this in the _calculate_blank_pyruvate() function
  * Measure NADH concentration
    * use the process_pdc_timecourse() function

* Combine the data into a single pandas dataframe for plotting
* Plot NADH concentration vs. offset time (i.e. where the assay start time has been shifted to zero) for all samples. This will allow us to do a rough examination of the data

* Data processing for subsequent analysis:
  * For each assay, measure the maximum slope (V), after the assay start.
  * Normalize V to the enzyme concentration (V/E)

* Determine the effect of Adh enzyme concentration
  * Select only the "Varying Adh" assay group
  * Plot V/E vs. the Adh concentration

* Create a kcat plot
  * Plot V/E vs. the substrate concentration
  * Adjust the units so that we can measure kcat directly from the plot
  * Color by filename

* Measure NADH degradation (see if we have good enough data for this)

* Convert to an EnzymeML file
* Upload EnzymeML file, colab notebook, and raw data to Janis Shin's github folder for subsequent modeling.




In [24]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys

# Add parent directory to path to import pda modules
parent_path = str(Path.cwd().parent.parent)
if parent_path not in sys.path:
    sys.path.insert(0, parent_path)

from pda.data_io import load_kinetic_data
from pda.spectral import calculate_concentrations
from pda.timecourse import process_pdc_timecourse

print("Setup complete! Imported all required modules.")
print("Autoreload enabled - modules will be reloaded automatically.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setup complete! Imported all required modules.
Autoreload enabled - modules will be reloaded automatically.


In [25]:
# Load metadata
metadata_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRVpwYqImFkaUigsWgrO9MRtWjYWwps82EExnomLqNr_hOUNViKF_fFyAhJfIqe3hDq0IEG76W4v_fO/pub?output=csv"
metadata_df = pd.read_csv(metadata_url)

# Load standards
standards_df = pd.read_csv("../spectra_data/NADH_Pyruvate_Standards.csv")

print(f"Loaded {len(metadata_df)} assays from metadata")
print(f"Loaded {len(standards_df)} wavelength points from standards")

Loaded 129 assays from metadata
Loaded 858 wavelength points from standards


In [26]:
# Filter for PDC forward assays that are not flagged to ignore
pdc_assays = metadata_df[
    (metadata_df['Assay'] == 'PDC_fwd') & 
    (metadata_df['Ignore'].isna())
].copy()

print(f"Found {len(pdc_assays)} PDC assays to process")
print(f"Assay groups: {pdc_assays['Assay Group'].unique()}")
print(f"Unique files: {pdc_assays['Filename'].nunique()}")

Found 96 PDC assays to process
Assay groups: ['Varying Adh' 'Varying pyr low NADH' 'Varying pyr high NADH']
Unique files: 32


In [27]:
# Create mapping from metadata filenames (.KD) to actual CSV files
assay_data_path = Path("../assay_data")

# Add CSV filename column
pdc_assays['csv_filename'] = pdc_assays['Filename'].str.replace('.KD', '.csv', regex=False)

# Check which files exist
pdc_assays['csv_exists'] = pdc_assays['csv_filename'].apply(
    lambda x: (assay_data_path / x).exists()
)

print(f"Total PDC assays: {len(pdc_assays)}")
print(f"CSV files found: {pdc_assays['csv_exists'].sum()}")
print(f"CSV files missing: {(~pdc_assays['csv_exists']).sum()}")

if (~pdc_assays['csv_exists']).any():
    print("Missing files:")
    print(pdc_assays[~pdc_assays['csv_exists']]['csv_filename'].unique())

# Filter to only assays with existing CSV files
pdc_assays = pdc_assays[pdc_assays['csv_exists']].copy()
print(f"Processing {len(pdc_assays)} assays with available CSV files")

Total PDC assays: 96
CSV files found: 66
CSV files missing: 30
Missing files:
['0116 1600 800 400MM PYR-1.csv' '0116 200 100 40MM PYR-2.csv'
 '0116 20 16 8 4MM PYR-4.csv' '0120 1600 800 400MM PYR-1.csv'
 '0120 200 100 40MM PYR-2.csv' '0120 20 16 8 4MM PYR-3.csv'
 '0121 1600 800 400MM PYR-1.csv' '0121 200 100 40MM PYR-2.csv'
 '0121 20 16 8 4MM PYR-3.csv']
Processing 66 assays with available CSV files


## Testing the effect of smoothing the standards

In [ ]:
# Create smoothed versions of standards using Savitzky-Golay and cubic spline
from scipy.signal import savgol_filter
from scipy.interpolate import UnivariateSpline

# Savitzky-Golay smoothing
# Window size: 51 (must be odd), polynomial order: 3
standards_savgol = standards_df.copy()
standards_savgol['NADH_Coeff'] = savgol_filter(standards_df['NADH_Coeff'], window_length=51, polyorder=3)
standards_savgol['PYR_Coeff'] = savgol_filter(standards_df['PYR_Coeff'], window_length=51, polyorder=3)

# Cubic spline smoothing
# s is the smoothing factor (larger = more smoothing)
standards_cubspl = standards_df.copy()

# Fit spline for NADH
nadh_spline = UnivariateSpline(standards_df['Wavelength'], standards_df['NADH_Coeff'], s=0.01)
standards_cubspl['NADH_Coeff'] = nadh_spline(standards_df['Wavelength'])

# Fit spline for Pyruvate
pyr_spline = UnivariateSpline(standards_df['Wavelength'], standards_df['PYR_Coeff'], s=0.01)
standards_cubspl['PYR_Coeff'] = pyr_spline(standards_df['Wavelength'])

print("Created smoothed standards:")
print(f"  Original:       {len(standards_df)} points")
print(f"  Savitzky-Golay: {len(standards_savgol)} points")
print(f"  Cubic Spline:   {len(standards_cubspl)} points")

## Debugging one dataset

In [ ]:
# Plot comparison of original vs smoothed standards
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('NADH Extinction Coefficients', 'Pyruvate Extinction Coefficients'),
    horizontal_spacing=0.12
)

# NADH subplot
fig.add_trace(
    go.Scatter(
        x=standards_df['Wavelength'],
        y=standards_df['NADH_Coeff'],
        mode='lines',
        name='Original',
        line=dict(color='gray', width=1),
        legendgroup='original',
        showlegend=True
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=standards_savgol['Wavelength'],
        y=standards_savgol['NADH_Coeff'],
        mode='lines',
        name='Savitzky-Golay',
        line=dict(color='blue', width=2),
        legendgroup='savgol',
        showlegend=True
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=standards_cubspl['Wavelength'],
        y=standards_cubspl['NADH_Coeff'],
        mode='lines',
        name='Cubic Spline',
        line=dict(color='red', width=2),
        legendgroup='cubspl',
        showlegend=True
    ),
    row=1, col=1
)

# Pyruvate subplot
fig.add_trace(
    go.Scatter(
        x=standards_df['Wavelength'],
        y=standards_df['PYR_Coeff'],
        mode='lines',
        name='Original',
        line=dict(color='gray', width=1),
        legendgroup='original',
        showlegend=False
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=standards_savgol['Wavelength'],
        y=standards_savgol['PYR_Coeff'],
        mode='lines',
        name='Savitzky-Golay',
        line=dict(color='blue', width=2),
        legendgroup='savgol',
        showlegend=False
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=standards_cubspl['Wavelength'],
        y=standards_cubspl['PYR_Coeff'],
        mode='lines',
        name='Cubic Spline',
        line=dict(color='red', width=2),
        legendgroup='cubspl',
        showlegend=False
    ),
    row=1, col=2
)

# Update axes
fig.update_xaxes(title_text="Wavelength (nm)", row=1, col=1)
fig.update_xaxes(title_text="Wavelength (nm)", row=1, col=2)
fig.update_yaxes(title_text="Extinction Coefficient (mM⁻¹ cm⁻¹)", row=1, col=1)
fig.update_yaxes(title_text="Extinction Coefficient (mM⁻¹ cm⁻¹)", row=1, col=2)

fig.update_layout(
    title_text="Comparison of Standards Smoothing Methods",
    height=500,
    showlegend=True,
    template='plotly_white'
)

fig.show()

In [ ]:
# Process test dataset with ORIGINAL standards
test_dataset_name = "0108 1600MM PYR -1.csv - CELL_1"

# Parse dataset name
parts = test_dataset_name.rsplit(' - ', 1)
csv_filename = parts[0]
cuvette = parts[1]

# Get test assay metadata
test_assay = pdc_assays[
    (pdc_assays['csv_filename'] == csv_filename) & 
    (pdc_assays['Cuvette'] == cuvette)
].iloc[0]

# Load data
csv_path = assay_data_path / test_assay['csv_filename']
spectral_df = load_kinetic_data(str(csv_path), sample_filter=test_assay['Cuvette'])

# Process with original standards
print("Processing with ORIGINAL standards...")
results_original = process_pdc_timecourse(
    spectral_df=spectral_df,
    standards_df=standards_df,
    assay_start_time=test_assay['Start_time_s'],
    blank_time=test_assay['Blank_time_s'],
    initial_pyruvate_mM=test_assay['Pyruvate_mM'],
    method='constrained',
    wavelength_range=(320, 420),
    absorbance_max=2,
    plot=False,
    verbose=False
)

print(f"✓ Processed {len(results_original)} time points with ORIGINAL standards")

In [ ]:
# Process test dataset with SAVITZKY-GOLAY smoothed standards
print("Processing with SAVITZKY-GOLAY smoothed standards...")
results_savgol = process_pdc_timecourse(
    spectral_df=spectral_df,
    standards_df=standards_savgol,
    assay_start_time=test_assay['Start_time_s'],
    blank_time=test_assay['Blank_time_s'],
    initial_pyruvate_mM=test_assay['Pyruvate_mM'],
    method='constrained',
    wavelength_range=(320, 420),
    absorbance_max=2,
    plot=False,
    verbose=False
)

print(f"✓ Processed {len(results_savgol)} time points with SAVITZKY-GOLAY standards")

In [ ]:
# Process test dataset with CUBIC SPLINE smoothed standards
print("Processing with CUBIC SPLINE smoothed standards...")
results_cubspl = process_pdc_timecourse(
    spectral_df=spectral_df,
    standards_df=standards_cubspl,
    assay_start_time=test_assay['Start_time_s'],
    blank_time=test_assay['Blank_time_s'],
    initial_pyruvate_mM=test_assay['Pyruvate_mM'],
    method='constrained',
    wavelength_range=(320, 420),
    absorbance_max=2,
    plot=False,
    verbose=False
)

print(f"✓ Processed {len(results_cubspl)} time points with CUBIC SPLINE standards")

In [ ]:
# Compare NADH vs Time for all three smoothing approaches
fig = go.Figure()

# Original standards
fig.add_trace(go.Scatter(
    x=results_original['Time_Relative_s'],
    y=results_original['NADH_mM'],
    mode='lines',
    name='Original Standards',
    line=dict(color='gray', width=2),
))

# Savitzky-Golay smoothed standards
fig.add_trace(go.Scatter(
    x=results_savgol['Time_Relative_s'],
    y=results_savgol['NADH_mM'],
    mode='lines',
    name='Savitzky-Golay Smoothed',
    line=dict(color='blue', width=2),
))

# Cubic spline smoothed standards
fig.add_trace(go.Scatter(
    x=results_cubspl['Time_Relative_s'],
    y=results_cubspl['NADH_mM'],
    mode='lines',
    name='Cubic Spline Smoothed',
    line=dict(color='red', width=2),
))

# Add vertical line at assay start
fig.add_vline(
    x=0,
    line_dash="dash",
    line_color="black",
    annotation_text="Assay Start",
    annotation_position="top"
)

fig.update_layout(
    title=f'Effect of Standards Smoothing on NADH Concentration<br>{test_dataset_name}',
    xaxis_title='Time from Assay Start (s)',
    yaxis_title='NADH Concentration (mM)',
    height=600,
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

# Print summary statistics
print("\nSummary Statistics (after assay start):")
print("="*60)

for name, results in [('Original', results_original), 
                       ('Savitzky-Golay', results_savgol), 
                       ('Cubic Spline', results_cubspl)]:
    after_start = results[results['Time_Relative_s'] >= 0]
    print(f"\n{name}:")
    print(f"  Mean NADH: {after_start['NADH_mM'].mean():.4f} mM")
    print(f"  Std NADH:  {after_start['NADH_mM'].std():.4f} mM")
    print(f"  Mean R²:   {after_start['R_squared'].mean():.6f}")

# Examine the sharp drop in NADH concentration around timepoint 61.5-62s
# Using the "0108 1600MM PYR -1.csv - CELL_1" dataset
# Times are relative to assay start

# Find the problem timepoints in the results (using Time_Relative_s)
problem_time_start = 61.5
problem_time_end = 62.0

# Look at results around this time
problem_window = results_original[
    (results_original['Time_Relative_s'] >= problem_time_start - 5) &
    (results_original['Time_Relative_s'] <= problem_time_end + 5)
].copy()

print("NADH Concentration around problem timepoint (relative to assay start):")
print("="*70)
print(problem_window[['Time_Relative_s', 'NADH_mM', 'R_squared', 'Intercept']].to_string(index=False))

# Calculate change in NADH between consecutive timepoints
problem_window['NADH_change'] = problem_window['NADH_mM'].diff()

print("\n\nNADH Changes (delta between consecutive points):")
print("="*70)
print(problem_window[['Time_Relative_s', 'NADH_mM', 'NADH_change', 'R_squared']].to_string(index=False))

# Identify largest drops
max_drop_idx = problem_window['NADH_change'].idxmin()
max_drop_row = problem_window.loc[max_drop_idx]

print(f"\n\nLargest drop:")
print(f"  At relative time: {max_drop_row['Time_Relative_s']:.1f} s (from assay start)")
print(f"  Absolute time: {max_drop_row['Time_s']:.1f} s")
print(f"  NADH change: {max_drop_row['NADH_change']:.6f} mM")
print(f"  R² at this point: {max_drop_row['R_squared']:.6f}")

In [ ]:
# Examine raw spectra at the problem timepoints
# Compare spectrum before, at, and after the drop

# Get spectral columns (wavelengths)
spectral_cols = [c for c in spectral_df.columns 
                 if c not in ['sample', 'Time_s', 'filename']]

# Convert relative times to absolute times
# Assay start time from metadata
assay_start = test_assay['Start_time_s']

# Relative times (from assay start)
rel_time_before = 61.4  # Just before the drop
rel_time_at = 62.0      # At the drop
rel_time_after = 62.4   # Just after the drop

# Convert to absolute times
time_before = assay_start + rel_time_before
time_at = assay_start + rel_time_at
time_after = assay_start + rel_time_after

# Find rows closest to our problem times
idx_before = (spectral_df['Time_s'] - time_before).abs().idxmin()
idx_at = (spectral_df['Time_s'] - time_at).abs().idxmin()
idx_after = (spectral_df['Time_s'] - time_after).abs().idxmin()

row_before = spectral_df.loc[idx_before]
row_at = spectral_df.loc[idx_at]
row_after = spectral_df.loc[idx_after]

print(f"Comparing raw spectra:")
print(f"  Before: t={row_before['Time_s']:.1f} s (relative: {row_before['Time_s']-assay_start:.1f} s)")
print(f"  At:     t={row_at['Time_s']:.1f} s (relative: {row_at['Time_s']-assay_start:.1f} s)")
print(f"  After:  t={row_after['Time_s']:.1f} s (relative: {row_after['Time_s']-assay_start:.1f} s)")

# Extract spectra in the wavelength range of interest (320-420 nm)
wavelengths = []
abs_before = []
abs_at = []
abs_after = []

for col in spectral_cols:
    try:
        wavelength = float(col)
        if 320 <= wavelength <= 420:
            wavelengths.append(wavelength)
            abs_before.append(row_before[col])
            abs_at.append(row_at[col])
            abs_after.append(row_after[col])
    except ValueError:
        continue

wavelengths = np.array(wavelengths)
abs_before = np.array(abs_before)
abs_at = np.array(abs_at)
abs_after = np.array(abs_after)

# Plot the raw spectra
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=wavelengths,
    y=abs_before,
    mode='lines',
    name=f'Before (t_rel={row_before["Time_s"]-assay_start:.1f}s)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=wavelengths,
    y=abs_at,
    mode='lines',
    name=f'At Drop (t_rel={row_at["Time_s"]-assay_start:.1f}s)',
    line=dict(color='red', width=2)
))

fig.add_trace(go.Scatter(
    x=wavelengths,
    y=abs_after,
    mode='lines',
    name=f'After (t_rel={row_after["Time_s"]-assay_start:.1f}s)',
    line=dict(color='green', width=2)
))

fig.update_layout(
    title='Raw Absorbance Spectra Around Problem Timepoint',
    xaxis_title='Wavelength (nm)',
    yaxis_title='Absorbance',
    height=500,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

# Calculate differences
diff_at_minus_before = abs_at - abs_before
diff_after_minus_at = abs_after - abs_at

print(f"\n\nSpectral differences:")
print(f"  Mean abs change (at - before): {np.mean(diff_at_minus_before):.6f}")
print(f"  Mean abs change (after - at):  {np.mean(diff_after_minus_at):.6f}")
print(f"  Std of (at - before): {np.std(diff_at_minus_before):.6f}")
print(f"  Std of (after - at):  {np.std(diff_after_minus_at):.6f}")

In [ ]:
# Examine raw spectra at the problem timepoints
# Compare spectrum before, at, and after the drop

# Get spectral columns (wavelengths)
spectral_cols = [c for c in spectral_df.columns 
                 if c not in ['sample', 'Time_s', 'filename']]

# Find rows closest to our problem times
time_before = 61.4  # Just before the drop
time_at = 62.0      # At the drop
time_after = 62.4   # Just after the drop

# Get the rows
idx_before = (spectral_df['Time_s'] - time_before).abs().idxmin()
idx_at = (spectral_df['Time_s'] - time_at).abs().idxmin()
idx_after = (spectral_df['Time_s'] - time_after).abs().idxmin()

row_before = spectral_df.loc[idx_before]
row_at = spectral_df.loc[idx_at]
row_after = spectral_df.loc[idx_after]

print(f"Comparing raw spectra:")
print(f"  Before: t={row_before['Time_s']:.1f} s")
print(f"  At:     t={row_at['Time_s']:.1f} s")
print(f"  After:  t={row_after['Time_s']:.1f} s")

# Extract spectra in the wavelength range of interest (320-420 nm)
wavelengths = []
abs_before = []
abs_at = []
abs_after = []

for col in spectral_cols:
    try:
        wavelength = float(col)
        if 320 <= wavelength <= 420:
            wavelengths.append(wavelength)
            abs_before.append(row_before[col])
            abs_at.append(row_at[col])
            abs_after.append(row_after[col])
    except ValueError:
        continue

wavelengths = np.array(wavelengths)
abs_before = np.array(abs_before)
abs_at = np.array(abs_at)
abs_after = np.array(abs_after)

# Plot the raw spectra
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=wavelengths,
    y=abs_before,
    mode='lines',
    name=f'Before (t={row_before["Time_s"]:.1f}s)',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=wavelengths,
    y=abs_at,
    mode='lines',
    name=f'At Drop (t={row_at["Time_s"]:.1f}s)',
    line=dict(color='red', width=2)
))

fig.add_trace(go.Scatter(
    x=wavelengths,
    y=abs_after,
    mode='lines',
    name=f'After (t={row_after["Time_s"]:.1f}s)',
    line=dict(color='green', width=2)
))

fig.update_layout(
    title='Raw Absorbance Spectra Around Problem Timepoint',
    xaxis_title='Wavelength (nm)',
    yaxis_title='Absorbance',
    height=500,
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

# Calculate differences
diff_at_minus_before = abs_at - abs_before
diff_after_minus_at = abs_after - abs_at

print(f"\n\nSpectral differences:")
print(f"  Mean abs change (at - before): {np.mean(diff_at_minus_before):.6f}")
print(f"  Mean abs change (after - at):  {np.mean(diff_after_minus_at):.6f}")
print(f"  Std of (at - before): {np.std(diff_at_minus_before):.6f}")
print(f"  Std of (after - at):  {np.std(diff_after_minus_at):.6f}")

In [ ]:
# Test if using fit_intercept=True solves the problem
# The intercept can absorb baseline shifts that would otherwise appear as concentration changes

# We need to modify process_pdc_timecourse to use fit_intercept=True
# For now, let's manually test calculate_concentrations on a problem spectrum

from pda.spectral import calculate_concentrations

# Get the spectrum at the problem timepoint
spectrum_at_drop = pd.DataFrame({
    'Wavelength': wavelengths,
    'Absorbance': abs_at
})

# Test with fit_intercept=False (current method)
result_no_intercept = calculate_concentrations(
    spectrum_df=spectrum_at_drop,
    standards_df=standards_df,
    wavelength_range=(320, 420),
    absorbance_max=2,
    fit_intercept=False,
    fixed_pyr=1342.86,  # Use the fixed pyruvate from the analysis
    plot=False
)

# Test with fit_intercept=True
result_with_intercept = calculate_concentrations(
    spectrum_df=spectrum_at_drop,
    standards_df=standards_df,
    wavelength_range=(320, 420),
    absorbance_max=2,
    fit_intercept=True,
    fixed_pyr=1342.86,  # Use the fixed pyruvate from the analysis
    plot=False
)

print("Comparison: fit_intercept effect on problem spectrum")
print("="*70)
print(f"\nWithout intercept (current method):")
print(f"  NADH: {result_no_intercept['NADH_Conc']:.6f} mM")
print(f"  Intercept: {result_no_intercept['Intercept']:.6f}")
print(f"  R²: {result_no_intercept['R_squared']:.6f}")

print(f"\nWith intercept:")
print(f"  NADH: {result_with_intercept['NADH_Conc']:.6f} mM")
print(f"  Intercept: {result_with_intercept['Intercept']:.6f}")
print(f"  R²: {result_with_intercept['R_squared']:.6f}")

print(f"\nDifference in NADH: {result_with_intercept['NADH_Conc'] - result_no_intercept['NADH_Conc']:.6f} mM")

In [ ]:
# Test fit_intercept=False (current default method)
print("Processing with fit_intercept=FALSE...")
results_no_intercept = process_pdc_timecourse(
    spectral_df=spectral_df,
    standards_df=standards_df,
    assay_start_time=test_assay['Start_time_s'],
    blank_time=test_assay['Blank_time_s'],
    initial_pyruvate_mM=test_assay['Pyruvate_mM'],
    method='constrained',
    wavelength_range=(320, 420),
    absorbance_max=2,
    fit_intercept=False,
    plot=True,
    verbose=False
)

print(f"\n✓ Processed {len(results_no_intercept)} time points with fit_intercept=False")

In [ ]:
# Test fit_intercept=True (allows baseline offset)
print("Processing with fit_intercept=TRUE...")
results_with_intercept = process_pdc_timecourse(
    spectral_df=spectral_df,
    standards_df=standards_df,
    assay_start_time=test_assay['Start_time_s'],
    blank_time=test_assay['Blank_time_s'],
    initial_pyruvate_mM=test_assay['Pyruvate_mM'],
    method='constrained',
    wavelength_range=(320, 420),
    absorbance_max=2,
    fit_intercept=True,
    plot=True,
    verbose=False
)

print(f"\n✓ Processed {len(results_with_intercept)} time points with fit_intercept=True")

In [ ]:
# Compare NADH traces: fit_intercept effect
fig = go.Figure()

# Without intercept
fig.add_trace(go.Scatter(
    x=results_no_intercept['Time_Relative_s'],
    y=results_no_intercept['NADH_mM'],
    mode='lines',
    name='fit_intercept=False',
    line=dict(color='red', width=2),
))

# With intercept
fig.add_trace(go.Scatter(
    x=results_with_intercept['Time_Relative_s'],
    y=results_with_intercept['NADH_mM'],
    mode='lines',
    name='fit_intercept=True',
    line=dict(color='blue', width=2),
))

# Add vertical line at assay start
fig.add_vline(
    x=0,
    line_dash="dash",
    line_color="black",
    annotation_text="Assay Start",
    annotation_position="top"
)

fig.update_layout(
    title=f'Effect of fit_intercept on NADH Jaggedness<br>{test_dataset_name}',
    xaxis_title='Time from Assay Start (s)',
    yaxis_title='NADH Concentration (mM)',
    height=600,
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

# Calculate and compare noise levels
after_start_no_int = results_no_intercept[results_no_intercept['Time_Relative_s'] >= 0]
after_start_with_int = results_with_intercept[results_with_intercept['Time_Relative_s'] >= 0]

print("\nComparison (after assay start):")
print("="*70)
print(f"\nfit_intercept=False:")
print(f"  Mean NADH: {after_start_no_int['NADH_mM'].mean():.6f} mM")
print(f"  Std NADH:  {after_start_no_int['NADH_mM'].std():.6f} mM")
print(f"  Mean R²:   {after_start_no_int['R_squared'].mean():.6f}")
print(f"  Mean Intercept: {after_start_no_int['Intercept'].mean():.6f}")

print(f"\nfit_intercept=True:")
print(f"  Mean NADH: {after_start_with_int['NADH_mM'].mean():.6f} mM")
print(f"  Std NADH:  {after_start_with_int['NADH_mM'].std():.6f} mM")
print(f"  Mean R²:   {after_start_with_int['R_squared'].mean():.6f}")
print(f"  Mean Intercept: {after_start_with_int['Intercept'].mean():.6f}")

print(f"\nReduction in noise (std):")
reduction = (after_start_no_int['NADH_mM'].std() - after_start_with_int['NADH_mM'].std()) / after_start_no_int['NADH_mM'].std() * 100
print(f"  {reduction:.1f}% reduction in NADH standard deviation")

In [29]:
# DEBUG: Test processing of a specific dataset
import traceback

# Specify the dataset to debug (copy from error messages)
dataset_name = "0108 1600MM PYR -1.csv - CELL_1"

# Parse dataset_name to extract csv_filename and cuvette
# Format: "{csv_filename} - {cuvette}"
parts = dataset_name.rsplit(' - ', 1)
csv_filename = parts[0]
cuvette = parts[1]

print(f"Parsed dataset_name: '{dataset_name}'")
print(f"  CSV filename: {csv_filename}")
print(f"  Cuvette: {cuvette}")
print()

# Get the specific assay from metadata
test_assay = pdc_assays[
    (pdc_assays['csv_filename'] == csv_filename) & 
    (pdc_assays['Cuvette'] == cuvette)
].iloc[0]

print(f"Testing: {test_assay['csv_filename']} - {test_assay['Cuvette']}")
print(f"Pyruvate: {test_assay['Pyruvate_mM']} mM")
print(f"Start: {test_assay['Start_time_s']} s")
print(f"Blank: {test_assay['Blank_time_s']} s")
print()

try:
    # Load the data
    csv_path = assay_data_path / test_assay['csv_filename']
    print(f"Loading from: {csv_path}")
    spectral_df = load_kinetic_data(str(csv_path), sample_filter=test_assay['Cuvette'])
    
    print(f"Loaded dataframe shape: {spectral_df.shape}")
    print(f"First few rows:")
    print(spectral_df.head())
    print()
    
    # Process timecourse
    print("Calling process_pdc_timecourse...")
    results = process_pdc_timecourse(
        spectral_df=spectral_df,
        standards_df=standards_df,
        assay_start_time=test_assay['Start_time_s'],
        blank_time=test_assay['Blank_time_s'],
        initial_pyruvate_mM=test_assay['Pyruvate_mM'],
        method='constrained',
        wavelength_range=(320, 420),
        absorbance_max=2,
        plot=True,
        verbose=True  # Turn on verbose for debugging
    )
    
    print(f"\n✓ Success! Processed {len(results)} time points")
    print(results.head())
    
except Exception as e:
    print(f"ERROR: {e}")
    print(f"Full traceback:")
    traceback.print_exc()

Parsed dataset_name: '0108 1600MM PYR -1.csv - CELL_1'
  CSV filename: 0108 1600MM PYR -1.csv
  Cuvette: CELL_1

Testing: 0108 1600MM PYR -1.csv - CELL_1
Pyruvate: 1600.0 mM
Start: 213.4 s
Blank: 82.4 s

Loading from: ..\assay_data\0108 1600MM PYR -1.csv
Loaded dataframe shape: (1877, 913)
First few rows:
   sample  Time_s       190       191       192       193       194       195  \
0  CELL_1     1.4  0.277165  0.306940  0.323261  0.308804  0.344403  0.362947   
1  CELL_1     1.9  0.296353  0.291461  0.319880  0.332403  0.338091  0.370307   
2  CELL_1     2.4  0.267727  0.306422  0.299278  0.333193  0.334060  0.372737   
3  CELL_1     2.9  0.301473  0.321259  0.319440  0.348073  0.348929  0.388478   
4  CELL_1     3.4  0.286538  0.306385  0.314200  0.332806  0.338448  0.368126   

        196       197  ...      1091      1092      1093      1094      1095  \
0  0.373945  0.395913  ...  0.027030  0.027008  0.025217  0.028226  0.025274   
1  0.388240  0.393983  ...  0.025543  0.025150


✓ Success! Processed 1877 time points
   Time_s  Time_Relative_s   NADH_mM  Pyruvate_mM  R_squared  Intercept  \
0     1.4           -212.0  0.022268  1342.863874   0.087286  -0.240026   
1     1.9           -211.5  0.022496  1342.863874   0.089640  -0.240096   
2     2.4           -211.0  0.021395  1342.863874   0.078540  -0.239809   
3     2.9           -210.5  0.022503  1342.863874   0.089811  -0.240132   
4     3.4           -210.0  0.022812  1342.863874   0.093923  -0.240223   

   Blank_Pyruvate_mM       Method  Fixed_Pyruvate  Assay_Start_s  Blank_Time_s  
0        1342.863874  constrained            True          213.4          82.4  
1        1342.863874  constrained            True          213.4          82.4  
2        1342.863874  constrained            True          213.4          82.4  
3        1342.863874  constrained            True          213.4          82.4  
4        1342.863874  constrained            True          213.4          82.4  


In [ ]:
# Process each PDC assay using process_pdc_timecourse()
import traceback

all_results = []
errors = []

for idx, assay in pdc_assays.iterrows():
    csv_path = assay_data_path / assay['csv_filename']
    
    print(f"Processing: {assay['csv_filename']} - {assay['Cuvette']}")
    print(f"  Pyruvate: {assay['Pyruvate_mM']} mM, Start: {assay['Start_time_s']} s, Blank: {assay['Blank_time_s']} s")
    
    try:
        # Load CSV file
        spectral_df = load_kinetic_data(str(csv_path), sample_filter=assay['Cuvette'])
        
        print(f"  Loaded {len(spectral_df)} rows for {assay['Cuvette']}")
        
        if len(spectral_df) == 0:
            msg = f"No data found for {assay['Cuvette']}"
            print(f"  WARNING: {msg}")
            errors.append({
                'filename': assay['csv_filename'],
                'cuvette': assay['Cuvette'],
                'error': msg
            })
            continue
        
        # Process timecourse
        results = process_pdc_timecourse(
            spectral_df=spectral_df,
            standards_df=standards_df,
            assay_start_time=assay['Start_time_s'],
            blank_time=assay['Blank_time_s'],
            initial_pyruvate_mM=assay['Pyruvate_mM'],
            method='constrained',
            wavelength_range=(320, 420),
            absorbance_max=2,
            plot=True,
            verbose=False
        )
        
        # Add metadata columns
        results['Filename'] = assay['Filename']
        results['csv_filename'] = assay['csv_filename']
        results['Cuvette'] = assay['Cuvette']
        results['Assay_Group'] = assay['Assay Group']
        results['Pdc_ug_ml'] = assay['Pdc_ug_ml']
        results['Adh_ug_ml'] = assay['Adh_ug_ml']
        
        all_results.append(results)
        
        print(f"  ✓ Processed {len(results)} time points")
        
    except Exception as e:
        msg = str(e)
        print(f"  ERROR: {msg}")
        print(f"  Full traceback:")
        traceback.print_exc()
        errors.append({
            'filename': assay['csv_filename'],
            'cuvette': assay['Cuvette'],
            'error': msg
        })
        continue

# Combine all results
if len(all_results) > 0:
    combined_df = pd.concat(all_results, ignore_index=True)
    print(f"{'='*60}")
    print(f"Successfully processed {len(all_results)} assays")
    print(f"Total time points: {len(combined_df)}")
    if len(errors) > 0:
        print(f"Errors encountered: {len(errors)}")
    print(f"{'='*60}")
else:
    print("No results to combine!")
    combined_df = pd.DataFrame()

# Show error summary if any
if len(errors) > 0:
    print(f"{'='*60}")
    print("ERROR SUMMARY:")
    print(f"{'='*60}")
    errors_df = pd.DataFrame(errors)
    print(errors_df.to_string())


In [ ]:
# Plot NADH concentration vs. time for all assays
combined_df['Assay_ID'] = combined_df['Cuvette'] + '_' + combined_df['csv_filename']

fig = go.Figure()

for assay_id in combined_df['Assay_ID'].unique():
    data = combined_df[combined_df['Assay_ID'] == assay_id]
    
    fig.add_trace(go.Scatter(
        x=data['Time_Relative_s'],
        y=data['NADH_mM'],
        mode='lines',
        name=assay_id,
        showlegend=True
    ))

fig.update_layout(
    title='NADH Concentration vs. Time (All PDC Assays)',
    xaxis_title='Time from Assay Start (s)',
    yaxis_title='NADH Concentration (mM)',
    height=600,
    hovermode='closest'
)

fig.show()

print(f"Plotted {combined_df['Assay_ID'].nunique()} assays")

In [ ]:
# Calculate V (maximum slope) and V/E for each assay
from scipy.stats import linregress

# Time window for initial rate calculation (seconds after assay start)
RATE_WINDOW_START = 0
RATE_WINDOW_END = 50

kinetic_results = []

for assay_id in combined_df['Assay_ID'].unique():
    assay_data = combined_df[combined_df['Assay_ID'] == assay_id].copy()
    
    # Filter to rate window
    rate_data = assay_data[
        (assay_data['Time_Relative_s'] >= RATE_WINDOW_START) &
        (assay_data['Time_Relative_s'] <= RATE_WINDOW_END)
    ]
    
    if len(rate_data) < 3:
        print(f"Warning: Not enough data points for {assay_id}")
        continue
    
    # Calculate slope using linear regression
    slope, intercept, r_value, p_value, std_err = linregress(
        rate_data['Time_Relative_s'],
        rate_data['NADH_mM']
    )
    
    # Get metadata
    metadata = assay_data.iloc[0]
    pdc_conc = metadata['Pdc_ug_ml']
    
    # Calculate V/E (normalize by enzyme concentration)
    if pd.notna(pdc_conc) and pdc_conc > 0:
        v_over_e = slope / pdc_conc
    else:
        v_over_e = np.nan
    
    kinetic_results.append({
        'Assay_ID': assay_id,
        'Filename': metadata['Filename'],
        'csv_filename': metadata['csv_filename'],
        'Cuvette': metadata['Cuvette'],
        'Assay_Group': metadata['Assay_Group'],
        'Pdc_ug_ml': pdc_conc,
        'Adh_ug_ml': metadata['Adh_ug_ml'],
        'Pyruvate_mM': metadata['Pyruvate_mM'],
        'V_mM_per_s': slope,
        'V_over_E': v_over_e,
        'Intercept_mM': intercept,
        'R_squared': r_value**2,
        'n_points': len(rate_data)
    })

kinetics_df = pd.DataFrame(kinetic_results)

print(f"Calculated kinetics for {len(kinetics_df)} assays")
print(f"Summary statistics for V/E:")
print(kinetics_df['V_over_E'].describe())
print(f"Assay groups: {kinetics_df['Assay_Group'].unique()}")

In [ ]:
# Plot individual assay timecourses with regression lines
n_plots = min(6, len(kinetics_df))
sample_assays = kinetics_df.head(n_plots)

for idx, row in sample_assays.iterrows():
    assay_id = row['Assay_ID']
    assay_data = combined_df[combined_df['Assay_ID'] == assay_id]
    
    # Create regression line points
    x_line = np.array([RATE_WINDOW_START, RATE_WINDOW_END])
    y_line = row['V_mM_per_s'] * x_line + row['Intercept_mM']
    
    # Create plot
    fig = go.Figure()
    
    # Add full timecourse
    fig.add_trace(go.Scatter(
        x=assay_data['Time_Relative_s'],
        y=assay_data['NADH_mM'],
        mode='markers',
        name='Data',
        marker=dict(size=4, color='blue')
    ))
    
    # Add regression line
    fig.add_trace(go.Scatter(
        x=x_line,
        y=y_line,
        mode='lines',
        name=f'Linear fit (V/E = {row["V_over_E"]:.6f})',
        line=dict(color='red', width=2)
    ))
    
    # Add shaded region for rate window
    fig.add_vrect(
        x0=RATE_WINDOW_START,
        x1=RATE_WINDOW_END,
        fillcolor='lightgray',
        opacity=0.2,
        line_width=0,
        annotation_text="Rate window",
        annotation_position="top left"
    )
    
    fig.update_layout(
        title=f'{row["Cuvette"]} - {row["csv_filename"]}<br>' + 
              f'V = {row["V_mM_per_s"]:.6f} mM/s, ' +
              f'V/E = {row["V_over_E"]:.6f} mM/s/(μg/mL), ' +
              f'R² = {row["R_squared"]:.4f}',
        xaxis_title='Time from Assay Start (s)',
        yaxis_title='NADH Concentration (mM)',
        height=400,
        showlegend=True
    )
    
    fig.show()

In [ ]:
# Plot V/E vs. ADH concentration for "Varying Adh" assay group
varying_adh = kinetics_df[kinetics_df['Assay_Group'] == 'Varying Adh'].copy()

if len(varying_adh) > 0:
    fig = go.Figure()
    
    # Add scatter points
    fig.add_trace(go.Scatter(
        x=varying_adh['Adh_ug_ml'],
        y=varying_adh['V_over_E'],
        mode='markers',
        marker=dict(
            size=10,
            color=varying_adh['R_squared'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title='R²'),
            line=dict(width=1, color='black')
        ),
        text=[f"{row['Cuvette']}<br>{row['csv_filename']}<br>R²={row['R_squared']:.3f}" 
              for _, row in varying_adh.iterrows()],
        hovertemplate='ADH: %{x:.3f} μg/mL<br>V/E: %{y:.6f}<br>%{text}<extra></extra>'
    ))
    
    fig.update_layout(
        title='Effect of ADH Concentration on PDC Reaction Rate<br>(Varying Adh Assay Group)',
        xaxis_title='ADH Concentration (μg/mL)',
        yaxis_title='V/E (mM/s per μg/mL PDC)',
        height=500,
        showlegend=False
    )
    
    fig.show()
    
    print(f"Varying Adh group statistics:")
    print(f"  Number of assays: {len(varying_adh)}")
    print(f"  ADH range: {varying_adh['Adh_ug_ml'].min():.3f} - {varying_adh['Adh_ug_ml'].max():.3f} μg/mL")
    print(f"  V/E range: {varying_adh['V_over_E'].min():.6f} - {varying_adh['V_over_E'].max():.6f}")
else:
    print("No 'Varying Adh' assays found in the dataset")